In [9]:
import sqlite3
import pandas as pd


In [10]:
# Read the code below and write your observation in the next cell

conn = sqlite3.connect(r'C:\Users\User\Downloads\DA\Internship\search_engine\data\eng_subtitles_database.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

[('zipfiles',)]


In [11]:
cursor.execute("PRAGMA table_info('zipfiles')")
cols = cursor.fetchall()
for col in cols:
    print(col[1])

num
name
content


In [12]:
df = pd.read_sql_query("""SELECT * FROM zipfiles""", conn)
df.head()

,num,name,content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...


In [13]:
import zipfile
import io

count = 0

def decode_method(binary_data):
    global count
    # Decompress the binary data using the zipfile module
    # print(count, end=" ")
    count += 1
    with io.BytesIO(binary_data) as f:
        with zipfile.ZipFile(f, 'r') as zip_file:
            # Assuming there's only one file in the ZIP archive
            subtitle_content = zip_file.read(zip_file.namelist()[0])
    
    # Now 'subtitle_content' should contain the extracted subtitle content
    return subtitle_content.decode('latin-1')  # Assuming the content is UTF-8 encoded text

In [14]:
from tqdm import tqdm
tqdm.pandas()

df['file_content'] = df['content'].progress_apply(decode_method)

# Display the first few rows of the DataFrame
df.head()

100%|███████████████████████████████████████████████████████████████████████████| 82498/82498 [01:48<00:00, 757.68it/s]


,num,name,content,file_content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."


In [15]:
df_sample= df.sample(frac=0.3)

import re
# Regular expression pattern to remove timestamps
timestamp_pattern = r'\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}'
# Regular expression pattern to remove unwanted characters like '\r', '\n', 'ï»¿'
unwanted_pattern = r'[\r\nï»¿]'

# Function to clean the content
def clean_content(text):
    text = re.sub(timestamp_pattern, '', text)  # Remove timestamps
    text = re.sub(unwanted_pattern, '', text)    # Remove unwanted characters
    text = re.sub(r'<.*?>', '', text)            # Remove HTML tags
    text = text.strip()                          # Strip leading/trailing whitespace
    text = re.sub(r'^\d+', '', text.strip())     # Remove leading '1'
    return text.strip()

# Apply the cleaning function to the 'content' column
df_sample['cleaned_content'] = df_sample['file_content'].progress_apply(clean_content) 

100%|███████████████████████████████████████████████████████████████████████████| 24749/24749 [02:21<00:00, 174.44it/s]


In [16]:
df_sample= df.sample(frac=0.1)

import re
# Regular expression pattern to remove timestamps
timestamp_pattern = r'\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}'
# Regular expression pattern to remove unwanted characters like '\r', '\n', 'ï»¿'
unwanted_pattern = r'[\r\nï»¿]'

# Function to clean the content
def clean_content(text):
    text = re.sub(timestamp_pattern, '', text)  # Remove timestamps
    text = re.sub(unwanted_pattern, '', text)    # Remove unwanted characters
    text = re.sub(r'<.*?>', '', text)            # Remove HTML tags
    text = text.strip()                          # Strip leading/trailing whitespace
    text = re.sub(r'^\d+', '', text.strip())     # Remove leading '1'
    return text.strip()

# Apply the cleaning function to the 'content' column
df_sample['cleaned_content'] = df_sample['file_content'].progress_apply(clean_content)



100%|█████████████████████████████████████████████████████████████████████████████| 8250/8250 [00:38<00:00, 214.05it/s]


In [17]:
## step3: Data Preprocessing on train data
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [18]:
df_sample

,num,name,content,file_content,cleaned_content
45674,9370832,the.king.of.queens.s08.e15.buggie.nights.(2006...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00s\x11\x9aV...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nAdver...",Advertise your product or brand herecontact ww...
68603,9463558,christy.(2023).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xacT\x9aV...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nAdvertis...",Advertise your product or brand herecontact ww...
27070,9289345,earthstorm.s01.e03.earthquake.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x84\xb4\x...,"1\r\n00:00:06,760 --> 00:00:11,720\r\n[rushing...",[rushing air]2[deep rumbling and thudding]3[sh...
4585,9201472,i.am.groot.s01.e03.groots.pursuit.(2022).eng.1cd,"b'PK\x03\x04\x14\x00\x00\x00\x08\x00,\x8d\x99V...","ï»¿1\r\n00:00:01,125 --> 00:00:03,125\r\n(MARV...",(MARVEL THEME MUSIC PLAYING)2(MUSIC FAST-FORWA...
74494,9488246,shameless.s11.e04.nimby.(2021).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x9b}\x9aV...,ï»¿[Script Info]\r\nTitle: Default file\r\nScr...,[Script Info]Title: Default fileScriptType: v4...
...,...,...,...,...,...
78248,9504798,queenmaker.s01.e07.episode.1.7.(2023).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xa9\x98\x...,"ï»¿1\r\n00:00:15,515 --> 00:00:17,684\r\n- Exc...","- Excuse me.- Get out of the way.2I'm sorry, b..."
9003,9219484,borsalino.(1970).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00t\x91\x99V...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch...",Watch any video online with Open-SUBTITLESFree...
22277,9270800,nova.s46.e14.the.planets.jupiter.(2019).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xc8\x9d\x...,"ï»¿1\r\n00:00:01,768 --> 00:00:05,034\r\nâªâ...",âªâª2ZACHARY QUINTO:Half a billion milesfrom...
8294,9216263,dexter.s01.e08.shrink.wrap.(2006).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\x90\x99V...,"ï»¿1\r\n00:00:01,005 --> 00:00:03,216\r\n<i>Pr...","Previously on Dexterâ¦2- Open up, Perry, it's..."


In [19]:
def preprocess(raw_text):
    # Removing special characters and digits
    sentence = re.sub("[^a-zA-Z]", " ", raw_text)

    #change sentence to lower case
    sentence = sentence.lower()

    # tokenize into words
    tokens =nltk.word_tokenize(sentence)

    #Lemmatize
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    #remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in lemmatized_tokens if word.lower() not in stop_words]

    #Join and return
    return " ".join(filtered_tokens)

In [20]:
df_sample['cleaned_content'] = df_sample['cleaned_content'].progress_apply(lambda x:  preprocess(x))

100%|██████████████████████████████████████████████████████████████████████████████| 8250/8250 [11:09<00:00, 12.33it/s]


In [21]:
df_sample

,num,name,content,file_content,cleaned_content
45674,9370832,the.king.of.queens.s08.e15.buggie.nights.(2006...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00s\x11\x9aV...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nAdver...",advertise product brand herecontact www opensu...
68603,9463558,christy.(2023).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xacT\x9aV...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nAdvertis...",advertise product brand herecontact www opensu...
27070,9289345,earthstorm.s01.e03.earthquake.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x84\xb4\x...,"1\r\n00:00:06,760 --> 00:00:11,720\r\n[rushing...",rushing air deep rumbling thudding shouting wo...
4585,9201472,i.am.groot.s01.e03.groots.pursuit.(2022).eng.1cd,"b'PK\x03\x04\x14\x00\x00\x00\x08\x00,\x8d\x99V...","ï»¿1\r\n00:00:01,125 --> 00:00:03,125\r\n(MARV...",marvel theme music playing music fast forward ...
74494,9488246,shameless.s11.e04.nimby.(2021).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x9b}\x9aV...,ï»¿[Script Info]\r\nTitle: Default file\r\nScr...,script info title default filescripttype v wra...
...,...,...,...,...,...
78248,9504798,queenmaker.s01.e07.episode.1.7.(2023).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xa9\x98\x...,"ï»¿1\r\n00:00:15,515 --> 00:00:17,684\r\n- Exc...",excuse get way sorry allowanyone inside campai...
9003,9219484,borsalino.(1970).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00t\x91\x99V...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch...",watch video online open subtitlesfree browser ...
22277,9270800,nova.s46.e14.the.planets.jupiter.(2019).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xc8\x9d\x...,"ï»¿1\r\n00:00:01,768 --> 00:00:05,034\r\nâªâ...",zachary quinto half billion milesfrom sun wind...
8294,9216263,dexter.s01.e08.shrink.wrap.(2006).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\x90\x99V...,"ï»¿1\r\n00:00:01,005 --> 00:00:03,216\r\n<i>Pr...",previously dexter open perry police open hey a...


In [22]:
def chunk_document(corpous, id_col, chunk_size=300):
    data = []

    for doc,id  in zip(corpous, id_col):
        
        words = doc.split()
        for i in range(0, len(words), chunk_size):
            chunk = ' '.join(words[i:i + chunk_size])


            data.append((id,chunk))

    df = pd.DataFrame(data)

    return df

In [23]:
df_sample

,num,name,content,file_content,cleaned_content
45674,9370832,the.king.of.queens.s08.e15.buggie.nights.(2006...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00s\x11\x9aV...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nAdver...",advertise product brand herecontact www opensu...
68603,9463558,christy.(2023).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xacT\x9aV...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nAdvertis...",advertise product brand herecontact www opensu...
27070,9289345,earthstorm.s01.e03.earthquake.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x84\xb4\x...,"1\r\n00:00:06,760 --> 00:00:11,720\r\n[rushing...",rushing air deep rumbling thudding shouting wo...
4585,9201472,i.am.groot.s01.e03.groots.pursuit.(2022).eng.1cd,"b'PK\x03\x04\x14\x00\x00\x00\x08\x00,\x8d\x99V...","ï»¿1\r\n00:00:01,125 --> 00:00:03,125\r\n(MARV...",marvel theme music playing music fast forward ...
74494,9488246,shameless.s11.e04.nimby.(2021).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x9b}\x9aV...,ï»¿[Script Info]\r\nTitle: Default file\r\nScr...,script info title default filescripttype v wra...
...,...,...,...,...,...
78248,9504798,queenmaker.s01.e07.episode.1.7.(2023).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xa9\x98\x...,"ï»¿1\r\n00:00:15,515 --> 00:00:17,684\r\n- Exc...",excuse get way sorry allowanyone inside campai...
9003,9219484,borsalino.(1970).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00t\x91\x99V...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch...",watch video online open subtitlesfree browser ...
22277,9270800,nova.s46.e14.the.planets.jupiter.(2019).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xc8\x9d\x...,"ï»¿1\r\n00:00:01,768 --> 00:00:05,034\r\nâªâ...",zachary quinto half billion milesfrom sun wind...
8294,9216263,dexter.s01.e08.shrink.wrap.(2006).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\x90\x99V...,"ï»¿1\r\n00:00:01,005 --> 00:00:03,216\r\n<i>Pr...",previously dexter open perry police open hey a...


In [24]:
chuncked_df = chunk_document(df_sample.cleaned_content,df_sample.num)

In [25]:
chuncked_df

,0,1
0,9370832,advertise product brand herecontact www opensu...
1,9370832,morning sa total party scene get totally waste...
2,9370832,see one u pin make sure sno illegal stuff like...
3,9370832,night really take back tomy junior college day...
4,9463558,advertise product brand herecontact www opensu...
...,...,...
75186,9471301,order mearound brass like leonard happens betw...
75187,9471301,lookover shoulder ever andyou world like used ...
75188,9471301,youcan blackmail wrong take eh think apuff goo...
75189,9471301,music sombre music continues sombre music cont...


In [26]:
chuncked_df.columns =['num','file_content_chunks']

In [27]:
chuncked_df

,num,file_content_chunks
0,9370832,advertise product brand herecontact www opensu...
1,9370832,morning sa total party scene get totally waste...
2,9370832,see one u pin make sure sno illegal stuff like...
3,9370832,night really take back tomy junior college day...
4,9463558,advertise product brand herecontact www opensu...
...,...,...
75186,9471301,order mearound brass like leonard happens betw...
75187,9471301,lookover shoulder ever andyou world like used ...
75188,9471301,youcan blackmail wrong take eh think apuff goo...
75189,9471301,music sombre music continues sombre music cont...


In [28]:
merged_df = chuncked_df.merge(df_sample,left_on="num",right_on="num",how="left")

In [29]:
merged_df

,num,file_content_chunks,name,content,file_content,cleaned_content
0,9370832,advertise product brand herecontact www opensu...,the.king.of.queens.s08.e15.buggie.nights.(2006...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00s\x11\x9aV...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nAdver...",advertise product brand herecontact www opensu...
1,9370832,morning sa total party scene get totally waste...,the.king.of.queens.s08.e15.buggie.nights.(2006...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00s\x11\x9aV...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nAdver...",advertise product brand herecontact www opensu...
2,9370832,see one u pin make sure sno illegal stuff like...,the.king.of.queens.s08.e15.buggie.nights.(2006...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00s\x11\x9aV...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nAdver...",advertise product brand herecontact www opensu...
3,9370832,night really take back tomy junior college day...,the.king.of.queens.s08.e15.buggie.nights.(2006...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00s\x11\x9aV...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nAdver...",advertise product brand herecontact www opensu...
4,9463558,advertise product brand herecontact www opensu...,christy.(2023).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xacT\x9aV...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nAdvertis...",advertise product brand herecontact www opensu...
...,...,...,...,...,...,...
75186,9471301,order mearound brass like leonard happens betw...,a.town.called.malice.s01.e08.episode.1.8.(2023...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x13j\x9aV...,"ï»¿1\r\n00:00:15,000 --> 00:00:17,679\r\nMINT ...",mint soproud alb letting boysspread wing dad b...
75187,9471301,lookover shoulder ever andyou world like used ...,a.town.called.malice.s01.e08.episode.1.8.(2023...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x13j\x9aV...,"ï»¿1\r\n00:00:15,000 --> 00:00:17,679\r\nMINT ...",mint soproud alb letting boysspread wing dad b...
75188,9471301,youcan blackmail wrong take eh think apuff goo...,a.town.called.malice.s01.e08.episode.1.8.(2023...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x13j\x9aV...,"ï»¿1\r\n00:00:15,000 --> 00:00:17,679\r\nMINT ...",mint soproud alb letting boysspread wing dad b...
75189,9471301,music sombre music continues sombre music cont...,a.town.called.malice.s01.e08.episode.1.8.(2023...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x13j\x9aV...,"ï»¿1\r\n00:00:15,000 --> 00:00:17,679\r\nMINT ...",mint soproud alb letting boysspread wing dad b...


In [30]:
merged_df = merged_df[['num','name', 'file_content_chunks']]	

In [31]:
merged_df

,num,name,file_content_chunks
0,9370832,the.king.of.queens.s08.e15.buggie.nights.(2006...,advertise product brand herecontact www opensu...
1,9370832,the.king.of.queens.s08.e15.buggie.nights.(2006...,morning sa total party scene get totally waste...
2,9370832,the.king.of.queens.s08.e15.buggie.nights.(2006...,see one u pin make sure sno illegal stuff like...
3,9370832,the.king.of.queens.s08.e15.buggie.nights.(2006...,night really take back tomy junior college day...
4,9463558,christy.(2023).eng.1cd,advertise product brand herecontact www opensu...
...,...,...,...
75186,9471301,a.town.called.malice.s01.e08.episode.1.8.(2023...,order mearound brass like leonard happens betw...
75187,9471301,a.town.called.malice.s01.e08.episode.1.8.(2023...,lookover shoulder ever andyou world like used ...
75188,9471301,a.town.called.malice.s01.e08.episode.1.8.(2023...,youcan blackmail wrong take eh think apuff goo...
75189,9471301,a.town.called.malice.s01.e08.episode.1.8.(2023...,music sombre music continues sombre music cont...


In [32]:
for id, chunk in enumerate(merged_df):
    print(chunk)

num
name
file_content_chunks


In [33]:
!pip install sentence_transformers

In [34]:

from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

C:\ana\envs\ana\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [35]:
pip install chromadb

Note: you may need to restart the kernel to use updated packages.


In [37]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import chromadb

In [38]:
merged_df

,num,name,file_content_chunks
0,9370832,the.king.of.queens.s08.e15.buggie.nights.(2006...,advertise product brand herecontact www opensu...
1,9370832,the.king.of.queens.s08.e15.buggie.nights.(2006...,morning sa total party scene get totally waste...
2,9370832,the.king.of.queens.s08.e15.buggie.nights.(2006...,see one u pin make sure sno illegal stuff like...
3,9370832,the.king.of.queens.s08.e15.buggie.nights.(2006...,night really take back tomy junior college day...
4,9463558,christy.(2023).eng.1cd,advertise product brand herecontact www opensu...
...,...,...,...
75186,9471301,a.town.called.malice.s01.e08.episode.1.8.(2023...,order mearound brass like leonard happens betw...
75187,9471301,a.town.called.malice.s01.e08.episode.1.8.(2023...,lookover shoulder ever andyou world like used ...
75188,9471301,a.town.called.malice.s01.e08.episode.1.8.(2023...,youcan blackmail wrong take eh think apuff goo...
75189,9471301,a.town.called.malice.s01.e08.episode.1.8.(2023...,music sombre music continues sombre music cont...


In [ ]:
# merged_df['doc_vector_pretrained_bert'] = merged_df.file_content_chunks.progress_apply(model.encode)

# merged_df.head()

 44%|████████████████████████████████                                        | 34095/76710 [1:51:54<1:44:20,  6.81it/s]

In [40]:
merged_df = pd.read_csv(r"C:\Users\User\Downloads\DA\Internship\search_engine\data\Sub_Titles_BERT.csv")

In [41]:
merged_df.head()

,num,name,file_content_chunks,doc_vector_pretrained_bert
0,9192535,mommy.issues.(2021).eng.1cd,place start door open good morning easy say sp...,[-7.83905983e-02 -1.13474317e-01 -4.56910208e-...
1,9199194,empire.s03.e04.cupid.kills.(2016).eng.1cd,direct result willful preventable malpractice ...,[-8.45821798e-02 8.84224102e-03 8.26942697e-...
2,9221632,this.is.us.s05.e15.jerry.2.0.(2021).eng.1cd,narrator previously beauty beast something gem...,[-7.01480880e-02 -6.54907450e-02 -3.89705948e-...
3,9209861,red.rose.s01.e04.manchester.innit.(2022).eng.1cd,mother hajara married bayo refused osage offer...,[-1.84880402e-02 3.30960639e-02 -4.40507084e-...
4,9209767,fruits.basket.s03.e11.goodbye.(2021).eng.1cd,dialogue dialogue unknown po head thing right ...,[ 2.88344882e-02 -4.04044986e-02 -5.03935665e-...


In [47]:
User_query = pd.Series([input("Enter a user Query " )])

Enter a user Query peter


In [44]:
from tqdm import tqdm, tqdm_notebook

In [45]:
tqdm.pandas()

In [48]:
User_query.progress_apply(lambda x:  preprocess(x, "lemma"))

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 443.33it/s]


TypeError: preprocess() takes 1 positional argument but 2 were given

In [43]:

User_query.progress_apply(lambda x:  clean_content(x)) 

User_query.progress_apply(lambda x:  preprocess(x)) 

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 200.49it/s]


0    peter
dtype: object

In [46]:
Query = User_query.progress_apply(model.encode)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 23.23it/s]


In [67]:
Query[0]

array([-2.16531549e-02,  4.10362966e-02, -1.03500835e-03, -2.97786091e-02,
       -6.63683042e-02, -9.52343550e-03,  5.62622100e-02,  3.15104425e-02,
       -2.29437146e-02, -3.30940373e-02, -6.69950396e-02, -4.52171043e-02,
       -9.01967287e-02, -3.01976991e-03, -1.97209488e-03, -1.07732890e-02,
       -7.68374801e-02,  5.28583005e-02,  1.70733035e-03, -9.42570642e-02,
       -1.29660293e-01, -1.79409180e-02,  2.80486345e-02, -4.87643257e-02,
        2.10065246e-02,  2.55819336e-02, -9.36805271e-03,  1.95052344e-02,
        1.09621994e-01, -1.16009109e-01, -1.70466397e-02,  8.89472850e-03,
       -2.86132880e-02, -5.01052178e-02, -2.82779653e-02, -6.98663585e-04,
       -6.15112968e-02,  1.11004747e-02, -2.52986941e-02,  1.76151283e-02,
        1.90327968e-02, -6.16267184e-03, -5.91798425e-02, -1.19332066e-02,
       -1.45981964e-02, -2.36388147e-02, -4.89224792e-02,  2.48159561e-02,
        5.96654937e-02,  4.84615862e-02, -4.98594046e-02, -1.48958955e-02,
        5.72910532e-04, -

In [86]:
cos_sim = util.cos_sim(Query, merged_df['doc_vector_pretrained_bert`'])


In [109]:
a=cos_sim.argsort()[:][::].tolist()

In [118]:
single_list = []
for sublist in a:
    single_list.extend(sublist)

In [119]:
top_sub = merged_df.iloc[sublist[::-1]]

In [122]:
k=int(input(" No_of docs you want"))
top_sub[:k]

 No_of docs you want 10


,num,name,file_content_chunks,doc_vector_pretrained_bert
166688,9279233,spiderman.no.way.home.(2021).eng.1cd,peter parker best friend come boy come flash t...,"[-0.070744194, -0.10539006, -0.013086834, -0.0..."
89256,9422450,family.guy.s14.e12.scammed.yankees.(2016).eng.1cd,peter go wait let get straight none guy haseve...,"[0.0108731985, -0.047444083, -0.08321992, -0.0..."
166701,9279233,spiderman.no.way.home.(2021).eng.1cd,yeah oh sorry mean peter peter called peter pe...,"[-0.034680914, -0.10299253, -0.007819386, -0.0..."
205958,9346037,heroes.s04.e17.chapter.seventeen.the.wall.(201...,peter previously hero lauren wait second clair...,"[-0.004613507, -0.05195206, 0.009521821, -0.08..."
33724,9418780,family.guy.s12.e15.secondhand.spoke.(2014).eng...,playing peter ready peter okay put peter wow w...,"[-0.04394487, -0.049517408, 0.026066598, -0.06..."
40000,9501633,love.me.s02.e04.episode.2.4.(2023).eng.1cd,wan na dosomething peter start showing upto ap...,"[0.014490434, -0.04982593, 0.022904452, -0.037..."
166690,9279233,spiderman.no.way.home.(2021).eng.1cd,peter owe pleasure right um really sorryto bot...,"[-0.08921423, -0.112601824, -0.037202526, -0.0..."
182176,9422394,family.guy.s11.e18.total.recall.(2013).eng.1cd,peter come time wake got get work ah lois feel...,"[-0.04550732, -0.099507734, 0.0032997, -0.0526..."
41207,9234383,the.real.charlie.chaplin.(2021).eng.1cd,saint peter mean old saint slammed gate right ...,"[0.010897958, 0.025107311, -6.395822e-05, -0.0..."
85909,9420951,family.guy.s08.e09.business.guy.(2009).eng.1cd,sir farting lois could youlet idiot husbandtak...,"[-0.006053679, -0.07671408, -0.0367564, -0.077..."


In [ ]:
merged_df.to_csv("Sub_Titles_with_32.csv",index=False)

In [49]:
merged_df['doc_vector_pretrained_bert'] = merged_df['doc_vector_pretrained_bert'].astype(str).str.replace("[","").str.replace("]","")

In [50]:
merged_df

,num,name,file_content_chunks,doc_vector_pretrained_bert
0,9192535,mommy.issues.(2021).eng.1cd,place start door open good morning easy say sp...,-7.83905983e-02 -1.13474317e-01 -4.56910208e-0...
1,9199194,empire.s03.e04.cupid.kills.(2016).eng.1cd,direct result willful preventable malpractice ...,-8.45821798e-02 8.84224102e-03 8.26942697e-0...
2,9221632,this.is.us.s05.e15.jerry.2.0.(2021).eng.1cd,narrator previously beauty beast something gem...,-7.01480880e-02 -6.54907450e-02 -3.89705948e-0...
3,9209861,red.rose.s01.e04.manchester.innit.(2022).eng.1cd,mother hajara married bayo refused osage offer...,-1.84880402e-02 3.30960639e-02 -4.40507084e-0...
4,9209767,fruits.basket.s03.e11.goodbye.(2021).eng.1cd,dialogue dialogue unknown po head thing right ...,2.88344882e-02 -4.04044986e-02 -5.03935665e-0...
...,...,...,...,...
7995,9209734,fruits.basket.s02.e03.shall.we.go.and.get.you....,working gotta go spent lot money least open ba...,-1.02519907e-01 -8.88609216e-02 3.08625363e-0...
7996,9194963,american.horror.stories.s02.e03.drive.(2022).e...,deeper ever seen black side town white side to...,-8.06297641e-04 -4.05393466e-02 -6.31514415e-0...
7997,9202976,doctor.of.doom.(1963).eng.1cd,tell july revolution remember date french revo...,-1.69950146e-02 -7.23221526e-02 9.06122755e-0...
7998,9204652,the.deuce.s02.e06.were.all.beasts.(2018).eng.1cd,kidding oh oh woe life empty without joy oh go...,-2.59164125e-02 -8.84233713e-02 3.32689956e-0...


In [52]:
a= merged_df['doc_vector_pretrained_bert'][:1]

In [54]:
Query = User_query.progress_apply(model.encode)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.96s/it]


In [71]:
merged_df['doc_vector_pretrained_bert'] = merged_df['doc_vector_pretrained_bert'].apply(lambda x: [float(i) for i in x.split()])

In [55]:
import torch

In [72]:
# Convert the DataFrame column into a list of tensors
doc_vectors = [torch.tensor(vec) for vec in merged_df['doc_vector_pretrained_bert']]

# Assuming Query[0] is also a tensor, convert it if necessary
query_tensor = torch.tensor(Query[0])

# Calculate cosine similarity
cos_similarities = [util.cos_sim(query_tensor, vec) for vec in doc_vectors]

In [73]:
client = chromadb.PersistentClient(path="Collections")

 collection = client.create_collection(
        name="search",
        metadata={"hnsw:space": "cosine"} # l2 is the default
    )

In [76]:
for i, emb in enumerate(doc_vectors):
    embs=np.array(emb)
    collection.add(ids=f"chunks_{i}",
    embeddings=embs.tolist(),
                   documents=merged_df.loc[i,"file_content_chunks"]
                  )


In [77]:
collection.get(include=["documents"])

{'ids': ['chunks_0',
  'chunks_1',
  'chunks_10',
  'chunks_100',
  'chunks_1000',
  'chunks_1001',
  'chunks_1002',
  'chunks_1003',
  'chunks_1004',
  'chunks_1005',
  'chunks_1006',
  'chunks_1007',
  'chunks_1008',
  'chunks_1009',
  'chunks_101',
  'chunks_1010',
  'chunks_1011',
  'chunks_1012',
  'chunks_1013',
  'chunks_1014',
  'chunks_1015',
  'chunks_1016',
  'chunks_1017',
  'chunks_1018',
  'chunks_1019',
  'chunks_102',
  'chunks_1020',
  'chunks_1021',
  'chunks_1022',
  'chunks_1023',
  'chunks_1024',
  'chunks_1025',
  'chunks_1026',
  'chunks_1027',
  'chunks_1028',
  'chunks_1029',
  'chunks_103',
  'chunks_1030',
  'chunks_1031',
  'chunks_1032',
  'chunks_1033',
  'chunks_1034',
  'chunks_1035',
  'chunks_1036',
  'chunks_1037',
  'chunks_1038',
  'chunks_1039',
  'chunks_104',
  'chunks_1040',
  'chunks_1041',
  'chunks_1042',
  'chunks_1043',
  'chunks_1044',
  'chunks_1045',
  'chunks_1046',
  'chunks_1047',
  'chunks_1048',
  'chunks_1049',
  'chunks_105',
  'c

In [78]:
doc_vectors[0]

tensor([-7.8391e-02, -1.1347e-01, -4.5691e-03, -5.6716e-03, -2.3426e-02,
         7.3255e-03, -5.6512e-03, -3.8667e-02, -3.6171e-02,  1.5680e-02,
         3.6550e-02, -1.0670e-02, -5.7361e-02, -4.9183e-02, -4.1457e-02,
        -2.9344e-02,  5.0526e-02,  3.5492e-02, -7.6502e-02,  4.5095e-02,
         2.6481e-02, -5.0474e-03,  4.9960e-02, -3.9480e-02, -1.0351e-01,
         4.6609e-02, -1.0493e-01, -6.3279e-04,  2.9298e-03, -3.3578e-02,
        -1.1555e-02,  9.6098e-02,  2.5953e-02,  3.7583e-02,  2.7453e-02,
         2.3105e-02, -1.8266e-02,  1.4465e-01,  3.6982e-02, -2.8531e-02,
        -5.5603e-02, -7.3799e-02, -7.9451e-03,  5.1660e-02, -3.8241e-02,
        -7.6856e-02, -3.3947e-02, -4.9130e-02,  4.0102e-02, -6.1319e-02,
        -3.9372e-02, -1.3360e-02,  3.0897e-02,  3.8811e-02, -3.8338e-02,
        -1.7315e-02,  3.6293e-04, -4.6049e-02, -3.7786e-02, -3.4589e-02,
        -5.0555e-02,  1.7924e-02, -4.3591e-03,  1.4208e-03, -3.1716e-02,
        -2.4761e-02, -1.1563e-02,  2.0041e-02,  4.5

In [79]:
df = merged_df.reset_index()

In [80]:
df

,index,num,name,file_content_chunks,doc_vector_pretrained_bert
0,0,9192535,mommy.issues.(2021).eng.1cd,place start door open good morning easy say sp...,"[-0.0783905983, -0.113474317, -0.00456910208, ..."
1,1,9199194,empire.s03.e04.cupid.kills.(2016).eng.1cd,direct result willful preventable malpractice ...,"[-0.0845821798, 0.00884224102, 0.0826942697, 0..."
2,2,9221632,this.is.us.s05.e15.jerry.2.0.(2021).eng.1cd,narrator previously beauty beast something gem...,"[-0.070148088, -0.065490745, -0.00389705948, 0..."
3,3,9209861,red.rose.s01.e04.manchester.innit.(2022).eng.1cd,mother hajara married bayo refused osage offer...,"[-0.0184880402, 0.0330960639, -0.0440507084, -..."
4,4,9209767,fruits.basket.s03.e11.goodbye.(2021).eng.1cd,dialogue dialogue unknown po head thing right ...,"[0.0288344882, -0.0404044986, -0.00503935665, ..."
...,...,...,...,...,...
7995,7995,9209734,fruits.basket.s02.e03.shall.we.go.and.get.you....,working gotta go spent lot money least open ba...,"[-0.102519907, -0.0888609216, 0.0308625363, -0..."
7996,7996,9194963,american.horror.stories.s02.e03.drive.(2022).e...,deeper ever seen black side town white side to...,"[-0.000806297641, -0.0405393466, -0.0631514415..."
7997,7997,9202976,doctor.of.doom.(1963).eng.1cd,tell july revolution remember date french revo...,"[-0.0169950146, -0.0723221526, 0.00906122755, ..."
7998,7998,9204652,the.deuce.s02.e06.were.all.beasts.(2018).eng.1cd,kidding oh oh woe life empty without joy oh go...,"[-0.0259164125, -0.0884233713, 0.0332689956, -..."
